We consider the diffusion problem

$$
    \begin{cases}
        \frac{\partial T}{\partial t} = \Delta T \quad &\text{ in } \Omega, \quad 0<t<t_\text{end}, \\
        T = T_0 \quad &\text{ in } \Omega, \quad t = 0, \\
        \nabla T \bullet \hat{\mathbf{n}} = 0 \quad &\text{ on } \partial\Omega, \quad 0<t<t_\text{end}
    \end{cases}
$$

on the domain $\Omega := [-r_x,r_x] \times [-r_y,r_y] \times [-r_z,r_z] \subset \mathbb{R}^3$. Here $\Delta$ is the Laplace operator
$$
    \Delta := \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2},
$$
$T_0$ is a given function
$$
    T_0 : \Omega \rightarrow \mathbb{R}
$$
and
$$
    \hat{\mathbf{n}}: \partial\Omega \rightarrow \mathbb{R}^3
$$
is the unit normal field to the domain boundary.

In [1]:
import torch
import NURBS_PyTorch_2 as NP2

In [ ]:
class Diffusion_box(NURBS_PyTorch_2.NURBS_object):
    
    def __init__(self,
                 *args,
                 **kwargs):
        super().__init__(n_inputs  = 4,
                         n_outputs = 1)
        
        